In [1]:
from pathlib import *
import numpy as np
import pandas as pd
import mne
import six, os
np.set_printoptions(linewidth=999999, precision=6, suppress=True, threshold=99999)
pd.set_option('display.width', 999999)
pd.set_option('display.max_colwidth', 300)
import nibabel as nib
import matplotlib.pyplot as plt
#plt.interactive(False)
from pylabs.correlation.atlas import mori_region_labels, JHUtracts_region_labels
from pylabs.projects.bbc.pairing import foster_behav_data, control_behav_data, behav_list
from pylabs.projects.bbc.correl.atlases import atlases
from pylabs.utils import run_subprocess, WorkingContext
from pylabs.utils.paths import getnetworkdataroot
fs = Path(getnetworkdataroot())
project = 'bbc'
statsdir = fs/project/'stats'/'py_correl_3rdpass'   #'py_correl_5thpass'
fname_templ = '{pool}_{mod}.nii'

In [2]:
cluster_fname = 'cluster_report.csv'
clusters = pd.DataFrame.from_csv(str(statsdir/cluster_fname))
clusters.rename(index=str, columns={' k': 'k', ' x': 'x', ' y': 'y', ' z': 'z', ' name': 'name', ' mori': 'mori', ' JHU_tracts': 'JHU_tracts'}, inplace=True)
clusters['mod'] = clusters['name'].str.split('-').apply(lambda x: x[-1])
clusters['pool'] = clusters['name'].str.split('-').apply(lambda x: x[-2])
clusters['dir'] = clusters['name'].str.split('-').apply(lambda x: x[-3])
clusters['behav'] = clusters['name'].str.split('-').apply(lambda x: x[:-3]).str.join('-')
clusters.head(15)

,k,x,y,z,name,mori,JHU_tracts,mod,pool,dir,behav
cluster-index,,,,,,,,,,,
201,16,76,41,35,PPVTSS-pos-foster-FA,CEREBELLUM,Cerebellum_L,FA,foster,pos,PPVTSS
208,21,74,38,36,CTOPPphoaCS-pos-foster-FA,CEREBELLUM,Cerebellum_L,FA,foster,pos,CTOPPphoaCS
69,41,36,45,36,CTOPPrnCS-pos-foster-FA,CEREBELLUM_WM_right,Cerebellum_R,FA,foster,pos,CTOPPrnCS
82,18,48,56,65,STIMQ-PSDSscaleScore1-to-15-SUM-neg-foster-FA,Splenium_of_corpus_callosum_right,mid_corpus_callosum,FA,foster,neg,STIMQ-PSDSscaleScore1-to-15-SUM
51,15,41,46,65,STIMQ-PSDSscaleScore1-to-15-SUM-neg-foster-FA,Splenium_of_corpus_callosum_right,Background,FA,foster,neg,STIMQ-PSDSscaleScore1-to-15-SUM
37,25,23,54,72,PPVTSS-pos-foster-MD,Genu_of_corpus_callosum_right,Background,MD,foster,pos,PPVTSS
313,23,76,50,57,PPVTSS-pos-foster-MD,Fornix(cres)_Stria_terminalis_left,Inferior_fronto_occipital_fasciculus_L,MD,foster,pos,PPVTSS
74,18,29,47,59,PPVTSS-pos-foster-MD,MIDDLE_TEMPORAL_GYRUS_right,Superior_longitudinal_fasciculus_R,MD,foster,pos,PPVTSS
161,42,42,30,78,CTOPPphoaCS-pos-foster-MD,SUPERIOR_PARIETAL_LOBULE_right,Background,MD,foster,pos,CTOPPphoaCS


In [6]:
data = pd.DataFrame(index=list(set(clusters['pool'].values)), columns=list(set(clusters['mod'].values)))
with WorkingContext(str(statsdir)):
    try:
        for pool in list(set(clusters['pool'].values)):
            for mod in list(set(clusters['mod'].values)):
                fname = fname_templ.format(pool=pool, mod=mod)
                data.set_value(pool, mod, nib.load(fname).get_data())
    except:
        print('an error has occured loading data into dataframe.')